In [ ]:
#Задача Тестирование методов класса

In [8]:
import os
import time
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from tqdm import tqdm

In [11]:
# Пути для сохранения изображений
BASE_DIR = r"D:\Эстер\с рс\Courses\ML Engineer\HWEster12"
DATASET_DIR = os.path.join(BASE_DIR, "dataset")

# Создаем основные папки, если их нет
os.makedirs(DATASET_DIR, exist_ok=True)

def setup_driver():
    """Настройка Selenium WebDriver."""
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")  # Запуск в фоновом режиме
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    return driver

def get_image_urls(query, num_images=1000):
    """Получаем URL-адреса изображений из Яндекс Картинок."""
    driver = setup_driver()
    search_url = f"https://yandex.ru/images/search?text={query}"
    driver.get(search_url)
    
    # Прокручиваем страницу для загрузки изображений
    for _ in range(20):
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
        time.sleep(2)
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    driver.quit()
    
    image_elements = soup.find_all('img')
    image_urls = []
    
    for img in image_elements:
        if 'src' in img.attrs:
            image_urls.append(urljoin("https://yandex.ru", img['src']))
        elif 'data-src' in img.attrs:
            image_urls.append(urljoin("https://yandex.ru", img['data-src']))
    
    return image_urls[:num_images]

def download_images(image_urls, folder_name):
    """Скачиваем изображения и сохраняем их с именами от 0000 до 0999."""
    folder_path = os.path.join(DATASET_DIR, folder_name)
    os.makedirs(folder_path, exist_ok=True)
    
    for idx, url in tqdm(enumerate(image_urls), total=len(image_urls)):
        file_name = f"{idx:04d}.jpg"  # Формат имени 0000.jpg, 0001.jpg и т. д.
        file_path = os.path.join(folder_path, file_name)
        
        try:
            response = requests.get(url, stream=True, timeout=10)
            if response.status_code == 200:
                with open(file_path, 'wb') as file:
                    for chunk in response.iter_content(1024):
                        file.write(chunk)
        except Exception as e:
            print(f"Ошибка загрузки {url}: {e}")


In [12]:
# Получаем ссылки на изображения
brown_bear_urls = get_image_urls("brown bear", num_images=1000)
polar_bear_urls = get_image_urls("polar bear", num_images=1000)

# Загружаем изображения в соответствующие папки
download_images(brown_bear_urls, "brown_bear")
download_images(polar_bear_urls, "polar_bear")


100%|██████████████████████████████████████████████████████████████████████████████████| 44/44 [00:07<00:00,  5.80it/s]
